In [41]:
import pulp

# Supply end: suppliers
supply = {
    "Supplier1": 20,
    "Supplier2": 30
}

# Demand end: consumers
demand = {
    "Consumer1": 25,
    "Consumer2": 15,
    "Consumer3": 10
}

# Cost info (per unit shipping qty)
costs = {
    ("Supplier1", "Consumer1"): 4,
    ("Supplier1", "Consumer2"): 3,
    ("Supplier1", "Consumer3"): 2,
    ("Supplier2", "Consumer1"): 5,
    ("Supplier2", "Consumer2"): 6,
    ("Supplier2", "Consumer3"): 8
}

# Minimization model
model = pulp.LpProblem("Transportation Problem", pulp.LpMinimize)

# Decision variables (routes) for determining shipping qty
routes = pulp.LpVariable.dicts("Route", costs, lowBound=0, cat='Continuous')

# Cost minimization
model += pulp.lpSum([routes[(s, d)] * costs[(s, d)] for (s, d) in costs]), "Total Transportation Cost"

# Supply constraint (practical limitation: shipping qty by a supplier <= available supply)
for s in supply:
    model += pulp.lpSum([routes[(s, d)] for d in demand]) <= supply[s], f"Supply Constraint {s}"

# Demand constraint (practical optimization: shipped qty to a consumer >= consumer demand)
for d in demand:
    model += pulp.lpSum([routes[(s, d)] for s in supply]) >= demand[d], f"Demand Constraint {d}"

# Solving the model
model.solve()

# Feasibility testing
if pulp.LpStatus[model.status] == "Optimal":
    # The optimum solution
    optimal_cost = pulp.value(model.objective)
    optimal_solution = [routes[(s, d)].varValue for (s, d) in costs]

    # Output the optimal solution
    print(f"Status: {pulp.LpStatus[model.status]}\n")
    print("\033[1mOptimal shipment plan:\033[0m")
    for (s, d) in costs:
        print(f"From {s} to {d}: {routes[(s, d)].varValue} units")
    print(f"\033[1mTotal transportation cost: {optimal_cost}\033[0m\n\n")
    

    # Exploring the alternatives (instant cross-checking)
    alternative_solutions = []
    perturb_amount = optimal_cost * 0.1  # alternative solutions calculated in 10% incremental cost

    # near-optimal solutions
    for i in range(1, 4):  # Exploring 3 alternative solutions only
        # Create a new model for each alternative solution
        alt_model = pulp.LpProblem(f"Alternative Transportation Problem {i}", pulp.LpMinimize)
        
        # Copy the decision variables
        alt_routes = pulp.LpVariable.dicts("Route", costs, lowBound=0, cat='Continuous')
        
        # Define the objective function for the alternative model
        alt_model += pulp.lpSum([alt_routes[(s, d)] * costs[(s, d)] for (s, d) in costs]), "Total Transportation Cost"
        
        # Copy the constraints
        for s in supply:
            alt_model += pulp.lpSum([alt_routes[(s, d)] for d in demand]) <= supply[s], f"Supply Constraint {s}"
        for d in demand:
            alt_model += pulp.lpSum([alt_routes[(s, d)] for s in supply]) >= demand[d], f"Demand Constraint {d}"
        
        # Add the perturbation constraint
        alt_model += pulp.lpSum([alt_routes[(s, d)] * costs[(s, d)] for (s, d) in costs]) >= optimal_cost + perturb_amount * i
        
        # Solve the alternative model
        alt_model.solve()

        # Storing the feasible solutions and their costs
        if pulp.LpStatus[alt_model.status] == "Optimal":
            solution = [alt_routes[(s, d)].varValue for (s, d) in costs]
            cost = pulp.value(alt_model.objective)
            alternative_solutions.append((solution, cost))
        else:
            print(f"No feasible solution found for alternative {i}")

    # Feasible near-optimums (cost only)
    for idx, (solution, cost) in enumerate(alternative_solutions):
        print(f"Nearest Alternative Solution {idx+1}; Cost: {cost: .1f}")
        print(f"Shipping Quantities: {solution}\n")
else:
    print("No feasible solution found for the initial problem.")

Status: Optimal

Optimal shipment plan:
From Supplier1 to Consumer1: 0.0 units
From Supplier1 to Consumer2: 10.0 units
From Supplier1 to Consumer3: 10.0 units
From Supplier2 to Consumer1: 25.0 units
From Supplier2 to Consumer2: 5.0 units
From Supplier2 to Consumer3: 0.0 units
Total transportation cost: 205.0


Nearest Alternative Solution 1; Cost:  225.5
Shipping Quantities: [1.1, 15.0, 3.9, 23.9, 0.0, 6.1]

Nearest Alternative Solution 2; Cost:  246.0
Shipping Quantities: [5.5, 14.5, 0.0, 19.5, 0.5, 10.0]

Nearest Alternative Solution 3; Cost:  266.5
Shipping Quantities: [15.75, 4.25, 0.0, 9.25, 10.75, 10.0]

